In [149]:
# libraries

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import requests
import re
from bs4 import BeautifulSoup
import lxml

from pymongo import MongoClient
from time import sleep

import hashlib
from fake_useragent import UserAgent

import chromedriver_binary
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import NoSuchElementException
from selenium.webdriver.support.expected_conditions import NoAlertPresentException
from selenium.webdriver.support.expected_conditions import NoSuchFrameException
from selenium.webdriver.support.expected_conditions import StaleElementReferenceException

In [ ]:
# MONGO CONNECTION :

client = MongoClient()
db_DD_Forecast_II = client['db_DD_Forecast_II']
# collect_city_evidence = db_DD_Forecast_II['collect_city_evidence']
collect_city_evidence2 = db_DD_Forecast_II['collect_city_evidence2']

In [ ]:
# OPEN THE DRIVER AND MAXIMIZE :

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')

driver = webdriver.Chrome(chrome_options=chrome_options)
driver.maximize_window()

> <h1>I :</h1>

##  CHECKING EVIDENCE OF CITIES  : -

In [ ]:
def seleniumBasedActivityScroll(driver):
    '''
    for selenium based page, perform some scroll Activity
    '''
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    driver.execute_script('window.scrollBy(0, -100);')
    driver.execute_script('window.scrollBy(0, 50);')
    driver.execute_script('window.scrollBy(0, -50);')
    sleep(2)

In [ ]:
def goBack(driver):
    
    while (True):
        driver.execute_script("window.history.go(-1)")
        sleep(4)
        soup = BeautifulSoup(driver.page_source, 'lxml')

        if len(re.findall(r"Find historical weather by searching for a city, zip code, or airport code. Include a date for which you would like to see weather history. You can select a range of dates in the results on the next page", str(soup))) > 0:
            break 

In [ ]:
# FOR EACH CITY, FIND THE EVIDENCE OF DATA TABLE EXISTENCE :

driver.get("https://www.wunderground.com/history/")

for i in range(0, total_cities):  # iterate through each city provided externally
    
    dict_evidence = {}
    
    each_city_india = listOfCities.loc[i, 'name'] # city to process (from csv)
    state_corresponding = listOfCities.loc[i, 'name_1']  # state of this city (from csv)
        
        
    if collect_city_evidence2.find_one({"city_from_csv":each_city_india.strip(), "state_from_csv":state_corresponding}):
        print("city already covered in the collection")
        continue
    
    
    # Open the driver 
    print("working for :","city from csv :", each_city_india, " state from csv :", state_corresponding, "opening url")
    
    sleep(5)
#     driver.get("https://www.wunderground.com/history/")
    driver.implicitly_wait(10) 
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    if len(re.findall(r"Error 404: Page Not Found", str(soup))) > 0 and len(re.findall(r"Either Brutus is in the server room or it might be that", str(soup))) > 0:
        sleep(1800) # sleep for 1/5 hours
        
        print("************************CLOSING*****************************")   
        #  driver.close()
        driver.quit()

        
    
    # else start exploring for this city 
    evidence_city_flag = 0
    
    each_city = each_city_india.strip()
    each_city_search_input = each_city + ", " + "india"  # input term to location search bar 
    
    # Set the year value to '2018':
    year_select = driver.find_element(By.XPATH, "//select[@class='year']//option[contains(text(),'2018')]")
    year_select.click()
    
#     # Set the month value to 'january':
#     month_select = driver.find_element(By.XPATH, "//select[@class='month']//option[contains(text(),'anuary')]")
#     month_select.click()

    
    location_input_bar = driver.find_element(By.XPATH, "//input[@placeholder='City, State or ZIP Code, or Airport Code' and @id='histSearch']")
    location_input_bar.clear()
    sleep(4)
    location_input_bar.send_keys(each_city_search_input)  # made an input of 'city name' to location bar
    sleep(9)
    location_input_bar.send_keys(Keys.DOWN)  # press down key only to select from the 1st option shown by site
    
    
    # WE need to verify whether the option provided by site contains term 'India' or not :    
    
    # prevent element staleness
    location_input_bar = driver.find_element(By.XPATH, "//input[@placeholder='City, State or ZIP Code, or Airport Code' and @id='histSearch']")
    proper_search_term_site_gives = location_input_bar.get_attribute('value')
    
    
    # Noting in dictionary :
    # 1.) test month
    # 2.) test day
    # 3.) test year
    
    dict_evidence['test month'] = driver.find_element(By.XPATH, r"//select[@class='month']//option[@selected='selected']").text
    dict_evidence['test day'] = driver.find_element(By.XPATH, r"//select[@class='day']//option[@selected='']").text
    dict_evidence['test year']= driver.find_element(By.XPATH, r"//select[@class='year']//option[contains(text(),'2018')]").text

    
    if len(re.findall(r"India", str(proper_search_term_site_gives))) > 0:
        # Contains 'India' term in input location search bar
        
                
        # Submit the information:
        submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @class='button radius' and @ value='Submit']")
        
        # AFTER SUBMIT BUTTON CLICKED, SEVERAL OPTIONS POSSIBLE :----
        submit_button.click()
        
        sleep(7) # let this city webpage open and load
        
        
        # CASE_A : AFTER 'SUBMIT' BUTTON CLICK, THE PAGE DOES NOT LOADS TO A NEW PAGE, INSTEAD ASKS FOR SELECT LOCATION :
        if str(driver.current_url).__contains__(r"https://www.wunderground.com/history/index.html?error=AMBIGUOUS"):
            select_a_location_text_existence = driver.find_element(By.XPATH, r"//div[@class='location-date-column']//p[@class='listHeading']").text 
            
            if len(re.findall(r"Select a location", select_a_location_text_existence)) > 0:
                
                under_select_a_location_names = driver.find_elements(By.XPATH, "//li//a[text()="  + "'" + proper_search_term_site_gives + "'"  +  "]" )
                
                # Ex. ['Rupnagar, India', 'Rupnagar, India']
                names_list = [each_name.text for each_name in under_select_a_location_names] 
                
                count_names = 0
                
                for each_name_in_list_names in names_list:
                    if proper_search_term_site_gives == each_name_in_list_names:
                        count_names += 1
                        
                if count_names == len(names_list):  
                    # CASE_A PROVEN TRUE, HENCE STORING THE LINKS :
                    
                    under_select_a_location_links = driver.find_elements(By.XPATH, "//li//a[text()="  + "'" + proper_search_term_site_gives + "'"  +  "]" )
                    links_list = [each_link_location.get_attribute('href') for each_link_location in under_select_a_location_links]
                    # NOT CHECKING EVIDENCE FOR DATA TIME BEING NOW, JUST STORING THE LINKS FOR CITY DATA PAGE
                    
                    dict_evidence['site_option_for_city_csv'] = proper_search_term_site_gives
                    dict_evidence['city url'] = links_list
                    dict_evidence['specical case'] = 1
                    dict_evidence['city_from_csv'] = each_city
                    dict_evidence['state_from_csv'] = state_corresponding
                    dict_evidence['evidence_city'] = 1
                    collect_city_evidence2.insert_one(dict_evidence)
                    continue        
        
        
        # CHECK FOR SERVER ERROR 404 :
        soup = BeautifulSoup(driver.page_source, 'lxml')
        if len(re.findall(r"Error 404: Page Not Found", str(soup))) > 0 and len(re.findall(r"Either Brutus is in the server room or it might be that", str(soup))) > 0:
            dict_evidence['site_option_for_city_csv'] = proper_search_term_site_gives
            dict_evidence['evidence_city'] = 0
            dict_evidence['city_from_csv'] = each_city
            dict_evidence['state_from_csv'] = state_corresponding
            dict_evidence['city url'] = driver.current_url
            dict_evidence['Error 404 '] = 1  # error 404 brutus in server room
            collect_city_evidence2.insert_one(dict_evidence)
            goBack(driver)
            sleep(5)
            
            continue
            
        
        # SEARCH PATTERN 1 FOR BS4 BASED DATA HEADING (FROM BS4 PERSPECTIVE):
        
        try:
            soup = BeautifulSoup(driver.page_source, 'lxml')
            sleep(2.5)
            
            dict_evidence['heading'] = re.sub(r'[\n\t]', '', soup.find('div', attrs={'class':'mainWrapper'}).find('div', attrs={'id':'location'}).h2.text).strip()
#             search_pattern_1 = soup.find('div', attrs={"id":"observations_details"}).table
            evidence_city_flag = 1
            dict_evidence['BS4 based'] = 1
#             dict_evidence['heading'] = driver.find_element(By.XPATH, "//div[@class='mainWrapper']//div[@id='location']//h2[1]").text.strip()
        except:
            # FROM BS4 PERSPECTIVE, HEADING WAS NOT FOUND, SO SEARCHING FOR HEADING FROM SELENIUM PERSPECTIVE :
            
            try:
                # SEARCH PATTERN 2 FOR SELENIUM BASED DATA HEADING (FROM SELENIUM PERSPECTIVE):
                sleep(8)          

                seleniumBasedActivityScroll(driver) # perform selenium based scroll activity
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                seleniumBasedActivityScroll(driver) # perform selenium based scroll activity twice
                driver.find_element_by_tag_name('body').send_keys(Keys.HOME)
                seleniumBasedActivityScroll(driver) # perform selenium based scroll activity twice
                sleep(5)
                soup = BeautifulSoup(driver.page_source, 'lxml')
                
                dict_evidence['heading'] = re.sub(r'[\n\t]','', soup.find('div', attrs={'class':'columns small-12'}).find('h1').text).strip()
#                 search_pattern_2 = soup.find('div', attrs={"class":"observation-table"}).table
                evidence_city_flag = 1
                dict_evidence['selenium based'] = 1 # requires selenium
#                 dict_evidence['heading'] = driver.find_element(By.XPATH, '//div[@class="columns small-12"]//h1').text.strip()

            except:
#                 # FROM SELENIUM PERSPECTIVE, DATA TABLE WAS NOT FOUND, SO SEARCHING FOR HEADING IF AVAILABLE

#                 try:
#                     dict_evidence['heading'] = driver.find_element(By.XPATH, '//div[@class="columns small-12"]//h1').text.strip()
#                     evidence_city_flag = 0
#                 except:

                # NEITHER BS4 (TABLE+HEADING) WORKED NOR SELENIUM (TABLE+HEADING) WORKED, 
                # no bs4 table and no bs4 heading
                # no selenium table and no selenium heading
                # completely broken case
                evidence_city_flag = 0

                        
        dict_evidence['site_option_for_city_csv'] = proper_search_term_site_gives
        dict_evidence['evidence_city'] = evidence_city_flag
        dict_evidence['city_from_csv'] = each_city
        dict_evidence['state_from_csv'] = state_corresponding
        dict_evidence['city url'] = driver.current_url
        
        # into the collection insert
        print("inserting.....")
        collect_city_evidence2.insert_one(dict_evidence)
        
        goBack(driver)
                        
        sleep(5)
        print('evidence is ', evidence_city_flag)
        
        
    else:
        # 'city name and location' search term that site gave didn't contain 'India'. 
        #  Conclusion: 1.) site rounded off to something else for our city input, maybe out of India OR 
        #              2.) site didn't show any options for our city from csv, hence might not contain city data at all.
        
#         evidence_city_flag = 0
        dict_evidence['site_option_for_city_csv'] = 'not_generated' # site dint suggest any options
        dict_evidence['evidence_city'] = 0
        dict_evidence['city_from_csv'] = each_city
        dict_evidence['state_from_csv'] = state_corresponding
        
        # into the collection insert
        collect_city_evidence2.insert_one(dict_evidence)
        
        
        # search term to location bar, site gave
        # each_city
        
    

In [ ]:
# Table targets we need to find as evidence :

# 1.) div_table_target = soup.find('div', attrs={"id":"observations_details"}).table  # BS4 BASED
# 2.) div_table_target = soup.find('div', attrs={"class":"observation-table"}).table  # SELENIUM BASED

> <h1>II :</h1>

> *<h1>SPECICAL CASES :*</h1> 

>+ SPECIAL CASES :

>+ WILL BE STORED IN DIFFERENT COLLECTION UNDER THE SAME DATABASE

In [ ]:
# MONGO CONNECTION :

client = MongoClient()
db_DD_Forecast_II = client['db_DD_Forecast_II']
collect_city_evidence2_specicalCases = db_DD_Forecast_II['collect_city_evidence2_specicalCases']

In [ ]:


for each_specical_case in collect_city_evidence2.find({'specical case':1}):
    
    dict_special_case = {}
    
    # These parameters are set already
    
    if collect_city_evidence2_specicalCases.find_one({"city_from_csv" : each_specical_case['city_from_csv'] , 'state_from_csv':each_specical_case['state_from_csv']}):
        # ALREADY COVERED THAT CITY FROM CSV
        print('city from csv already covered')
        continue
     
    dict_special_case['test month'] = each_specical_case['test month'] 
    dict_special_case['test day'] = each_specical_case['test day'] 
    dict_special_case['test year'] = each_specical_case['test year'] 
    dict_special_case['site_option_for_city_csv'] = each_specical_case['site_option_for_city_csv'] 
    dict_special_case['city_from_csv'] = each_specical_case['city_from_csv']
    dict_special_case['state_from_csv'] = each_specical_case['state_from_csv']
    dict_special_case['city url'] = each_specical_case['city url'] 
    
    
    for count, each_url in enumerate(each_specical_case['city url']):
        
        
        dict_special_case['city_url_' + str(count)] = each_url
                
        # REQUEST THIS URL
        ua = UserAgent()
        header = {'User-Agent':str(ua.chrome)}
        responseCityUrl = requests.get(each_url, headers=header)
        sleep(4)
        soup = BeautifulSoup(responseCityUrl.text, 'lxml')
        sleep(2)
        
        
        # CHECK FOR SERVER ERROR 404 :
        if len(re.findall(r"Error 404: Page Not Found", str(soup))) > 0 and len(re.findall(r"Either Brutus is in the server room or it might be that", str(soup))) > 0:
            dict_special_case['Error 404_' + str(count)] = 1
            dict_special_case['BS4 based_' + str(count)] = 0
            dict_special_case['selenium based_' + str(count)] = 0 
            continue
            
        
        try:
            # SEARCHING FOR BS4 BASED HEADING
            # IF BS4 BASED HEADING IS FOUND, THEN THE DATA EXISTS
            
            dict_special_case['heading_' + str(count)] = re.sub(r'[\n\t]', '', soup.find('div', attrs={'class':'mainWrapper'}).find('div', attrs={'id':'location'}).h2.text).strip()
            dict_special_case['BS4 based_' + str(count)] = 1
            dict_special_case['selenium based_' + str(count)] = 0
            dict_special_case['evidence_city_' + str(count)] = 1
            
        except:
            # BS4 BASED HEADING WAS NOT FOUND , 
            # SO SEARCHING FOR SELENIUM BASED HEADING 
            
            try:
                dict_special_case['heading_' + str(count)] = re.sub(r'[\n\t]','', soup.find('div', attrs={'class':'columns small-12'}).find('h1').text).strip()
                dict_special_case['BS4 based_' + str(count)] = 0
                dict_special_case['selenium based_' + str(count)] = 1
                dict_special_case['evidence_city_' + str(count)] = 1
                
            except:
                # NEITHER BS4 BASED NOR SELENIUM BASED HEADING WAS FOUND
                # HENCE POSSIBILITIES ARE 
                # 1.) ERROR 404 
                # 2.) SITE DIDN'T RESPOND VIA REQUESTS MODULE PYTHON
                
                dict_special_case['BS4 based_' + str(count)] = 0
                dict_special_case['selenium based_' + str(count)] = 0
                dict_special_case['evidence_city_' + str(count)] = 0
                dict_special_case['unknown error_' + str(count)] = 1

                
        # INSERTING INTO A SEPARATE COLLECTION NAMED 'collect_city_evidence2_specicalCases' : 
        
#         uniqueStringToEncode = each_specical_case['city_from_csv'] + each_specical_case['state_from_csv'] 
#         mongoIdHash = hashlib.md5(uniqueStringToEncode.encode('utf-8')).hexdigest()        
        
        
#         dict_special_case['_id'] = mongoIdHash
        
    collect_city_evidence2_specicalCases.insert_one(dict_special_case)